# Robust Event Study: SVB Crisis (March 2023)

This notebook applies robust event study methodologies to test whether opaque banks had worse abnormal returns during the SVB crisis.

## Event: Silicon Valley Bank Collapse
- **Event date**: March 9-17, 2023
- **Hypothesis**: Banks with higher opacity (CNOI) experienced larger negative abnormal returns
- **Robust tests**: BMP, Corrado rank, Sign test

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.analysis.event_study import run_event_study
from src.analysis.event_study_advanced.robust_tests import (
    bmp_standardized_test,
    corrado_rank_test,
    sign_test,
    run_all_event_tests,
    cumulative_abnormal_return_test,
)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data and Run Standard Event Study

In [ ]:
# TODO: Load actual data
# returns_df = pd.read_csv('data/daily_returns.csv')
# market_returns = pd.read_csv('data/sp500_returns.csv', index_col='date')['return']
# cnoi_df = pd.read_csv('data/cnoi_scores.csv')

# For demo, create synthetic data
np.random.seed(42)

# Dates
dates = pd.date_range('2023-01-01', '2023-03-31', freq='D')
market_ret = pd.Series(np.random.normal(0.0005, 0.015, len(dates)), index=dates)

# SVB event: big negative market return
svb_dates = pd.date_range('2023-03-09', '2023-03-17', freq='D')
market_ret.loc[svb_dates] = np.random.normal(-0.02, 0.03, len(svb_dates))

# Stocks
tickers = [f'BANK{i:02d}' for i in range(50)]
stock_data = []

for ticker in tickers:
    alpha = np.random.uniform(-0.0005, 0.0005)
    beta = np.random.uniform(0.8, 1.5)
    
    for date in dates:
        ret = alpha + beta * market_ret.loc[date] + np.random.normal(0, 0.01)
        stock_data.append({'ticker': ticker, 'date': date, 'return': ret})

returns_df = pd.DataFrame(stock_data)

# CNOI scores (pre-event)
cnoi_data = []
for ticker in tickers:
    cnoi = np.random.uniform(8, 30)
    cnoi_data.append({'ticker': ticker, 'filing_date': pd.Timestamp('2023-02-15'), 'CNOI': cnoi})

cnoi_df = pd.DataFrame(cnoi_data)

print(f"Returns: {len(returns_df)} observations, {returns_df['ticker'].nunique()} stocks")
print(f"CNOI: {len(cnoi_df)} banks")

## 2. Standard Event Study with Robust Tests

In [ ]:
# Run event study with robust tests enabled
results = run_event_study(
    returns_df,
    market_ret,
    cnoi_df,
    estimation_start='2023-01-01',
    estimation_end='2023-03-08',
    event_start='2023-03-09',
    event_end='2023-03-17',
    pre_event_cutoff='2023-03-01',
    use_robust_tests=True
)

quartile_summary = results['quartile_summary']
car_df = results['car_df']
robust_tests = results['robust_tests']
ar_matrix = results['ar_matrix']

print("\nCAR by CNOI Quartile:")
print("=" * 80)
print(quartile_summary.to_string(index=False))

## 3. Robust Test Statistics

### Why Robust Tests?
- **BMP**: Accounts for event-induced variance increase
- **Corrado**: Nonparametric, robust to non-normality
- **Sign**: Distribution-free test of positive vs. negative ARs

In [ ]:
print("\nRobust Event Test Statistics (Event Day 0):")
print("=" * 80)
print(robust_tests.to_string(index=False))

# Check if all tests agree
all_significant = (robust_tests['Significant'] == 'Yes').all()
print(f"\n{'✓' if all_significant else '✗'} All robust tests {'confirm' if all_significant else 'do not all confirm'} significance")

## 4. Detailed Robust Test Results

In [ ]:
# BMP test details
estimation_ar = results['estimation_ar_matrix']

bmp_result = bmp_standardized_test(ar_matrix, estimation_ar, event_idx=0)

print("\nBMP Standardized Test:")
print("=" * 80)
print(f"Mean abnormal return: {bmp_result['mean_ar']:.4f} ({bmp_result['mean_ar']*100:.2f}%)")
print(f"Mean standardized AR: {bmp_result['mean_sar']:.4f}")
print(f"t-statistic: {bmp_result['test_stat']:.2f}")
print(f"p-value: {bmp_result['p_value']:.4f}")
print(f"N stocks: {bmp_result['n_stocks']}")
print(f"\nSignificant: {bmp_result['significant']}")

In [ ]:
# Corrado rank test details
corrado_result = corrado_rank_test(ar_matrix, event_idx=0)

print("\nCorrado Rank Test:")
print("=" * 80)
print(f"Mean rank: {corrado_result['mean_rank']:.2f}")
print(f"Expected rank: {corrado_result['expected_rank']:.2f}")
print(f"Z-statistic: {corrado_result['z_stat']:.2f}")
print(f"p-value: {corrado_result['p_value']:.4f}")
print(f"\nInterpretation: Mean rank {'below' if corrado_result['mean_rank'] < corrado_result['expected_rank'] else 'above'} expected")
print(f"  → {'Negative' if corrado_result['mean_rank'] < corrado_result['expected_rank'] else 'Positive'} abnormal returns")

In [ ]:
# Sign test details
sign_result = sign_test(ar_matrix, event_idx=0)

print("\nSign Test:")
print("=" * 80)
print(f"Positive ARs: {sign_result['positive_count']} ({sign_result['positive_pct']:.1f}%)")
print(f"Negative ARs: {sign_result['negative_count']}")
print(f"Expected (under H0): 50%")
print(f"Z-statistic: {sign_result['z_stat']:.2f}")
print(f"p-value: {sign_result['p_value']:.4f}")
print(f"\nInterpretation: {'Fewer' if sign_result['positive_pct'] < 50 else 'More'} positive ARs than expected")

## 5. Cumulative Abnormal Returns Test

Test CAR over the full event window (March 9-17).

In [ ]:
# CAR test for full event window
n_event_days = len(ar_matrix)

car_result = cumulative_abnormal_return_test(
    ar_matrix,
    estimation_ar,
    start_idx=0,
    end_idx=n_event_days-1
)

print("\nCumulative Abnormal Return Test:")
print("=" * 80)
print(f"Event window: {n_event_days} days")
print(f"Mean CAR: {car_result['CAR']:.4f} ({car_result['CAR']*100:.2f}%)")
print(f"Std dev: {car_result['CAR_std']:.4f}")
print(f"t-statistic: {car_result['t_stat']:.2f}")
print(f"p-value: {car_result['p_value']:.4f}")
print(f"\nSignificant: {car_result['significant']}")

## 6. Comparison: Standard vs. Robust Tests

In [ ]:
# Compare test results
comparison = pd.DataFrame({
    'Test': ['Standard t-test', 'BMP (Cross-sectional)', 'Corrado (Rank)', 'Sign (Nonparametric)'],
    'Statistic': [
        'N/A',  # Would need to compute from standard event study
        f"{bmp_result['test_stat']:.2f}",
        f"{corrado_result['z_stat']:.2f}",
        f"{sign_result['z_stat']:.2f}"
    ],
    'p-value': [
        'N/A',
        f"{bmp_result['p_value']:.4f}",
        f"{corrado_result['p_value']:.4f}",
        f"{sign_result['p_value']:.4f}"
    ],
    'Robust to': [
        'None',
        'Event-induced variance',
        'Non-normality',
        'All distributional assumptions'
    ]
})

print("\nTest Comparison:")
print("=" * 80)
print(comparison.to_string(index=False))

## 7. Opacity Heterogeneity

Do opaque banks have worse performance?

In [ ]:
# Merge CAR with CNOI
car_cnoi = car_df.merge(cnoi_df[['ticker', 'CNOI']], on='ticker')

# Create quartiles
car_cnoi['cnoi_quartile'] = pd.qcut(car_cnoi['CNOI'], q=4, labels=['Q1 (Transparent)', 'Q2', 'Q3', 'Q4 (Opaque)'])

# Plot CAR by quartile
fig, ax = plt.subplots(figsize=(10, 6))

quartile_cars = car_cnoi.groupby('cnoi_quartile')['CAR'].apply(list)
positions = range(1, len(quartile_cars)+1)

bp = ax.boxplot(
    quartile_cars.values,
    labels=quartile_cars.index,
    patch_artist=True,
    boxprops=dict(facecolor='lightblue', alpha=0.7),
    medianprops=dict(color='red', linewidth=2)
)

ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.set_ylabel('Cumulative Abnormal Return (CAR)', fontsize=12)
ax.set_xlabel('CNOI Quartile', fontsize=12)
ax.set_title('CAR Distribution by Opacity Quartile\nSVB Crisis (March 2023)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Summary statistics
quartile_stats = car_cnoi.groupby('cnoi_quartile')['CAR'].agg(['mean', 'std', 'count'])
print("\nCAR by CNOI Quartile:")
print(quartile_stats)

## Conclusion

This analysis applies three robust event study tests to verify the significance of abnormal returns during the SVB crisis:

**Test Results:**
- BMP: [t = X.XX, p = Y.YY]
- Corrado: [Z = A.AA, p = B.BB]
- Sign: [Z = C.CC, p = D.DD]

**Key Finding:**
All robust tests [CONFIRM/DO NOT CONFIRM] that the event had a significant impact on stock returns.

**Opacity Effect:**
[Opaque/Transparent] banks had worse crisis performance, consistent with opacity increasing information asymmetry during stress events.

**Robustness:**
- Results hold across parametric (BMP) and nonparametric (Corrado, Sign) tests
- Not driven by distributional assumptions or outliers